# Clustering and segmenting neigborhood in Paris 

    Liman M

1- Introduction

1.1- Les Faits

Paris est connue dans le monde entier pour ses monuments et sa vie artistique et culturelle, Paris est aussi une ville importante dans l’histoire du monde, avec un impact politique et économique majeur. Symbole de la culture française, son animation et ses grands musées en font une attraction pour près de trente millions de visiteurs par an. Paris est également considérée comme une des capitales mondiales de la mode et du luxe.

Paris est la neuvième ville la plus chère du monde en ce qui concerne les prix de l'immobilier de luxe : 12 600 euros/m2 en 2007 (contre 36 800 pour Londres, la plus chère)84,85. Selon une enquête réalisée pour le journal La Tribune, au 1er septembre 2012, la rue la plus chère de Paris est le quai des Orfèvres (1er arr.), avec un prix médian de 20 665 euros/m2, contre 3 900 euros/m2 rue Pajol (18e)86. Paris devient en 2017 la ville la plus attractive d'Europe pour les investisseurs immobiliers, ce qui ne s'était pas produit depuis 2007. Une étude de l'Institut d'aménagement et d'urbanisme (IAU) publiée en 2019 souligne que les prix du logement poussent les personnes modestes à quitter Paris et à s'installer dans des départements limitrophes comme la Seine-Saint-Denis, ce qui tend à provoquer un « embourgeoisement » de la capitale et une paupérisation des proches départements.

1-2- Problématique et intérêts

Il s'agit de prédire pour un investisseur immobilier, les lieux attractives de paris ou il est plus avantageux d'investir.

2- Données et méthodes de résolutions

La page wikipédédia et Foursquare webpage seront les bases de données qui seront extraits, restructuré pour analysé et comparer les données selon les arrondissements.

3- Exploitation des données 

In [1]:
import requests
import csv
import lxml.html as lh
from bs4 import BeautifulSoup
import pandas as pd
from arcgis.gis import GIS
import urllib
from geopy.geocoders import Nominatim
import folium # map rendering library
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib as plt
import numpy as np
from pandas.io.json import json_normalize
import wget
from sklearn.preprocessing import MinMaxScaler

ModuleNotFoundError: No module named 'folium'

3-1- Utilisé Beautiful Soup's library functions pour extraire Arrondissement(Arr.), Nom, Population et densité from Wikipedia page

In [2]:
URL = 'https://fr.wikipedia.org/wiki/Arrondissements_de_Paris?veaction=edit&section=2'
page = requests.get(URL)
html = BeautifulSoup(page.text, 'html.parser')
table = html.findAll('table',{"class":"sortable"})[0]
end_col = len(table.findAll('th'))
end_row = len(table.findAll('tr'))

3-2-Extraire les données avec html tags 

In [12]:
row_data=[]
start_col = 0
start_row = 2   
tr = table.findAll(['tr'])[start_row:end_row]
th = table.find_all(['th'])[start_col:end_col]
th_data = [col.text.strip('\n') for col in th]
for cell in tr:
    td = cell.find_all('td')
    row = [i.text.replace('\n','') for i in td]
    if row[2] != "Not assigned":
        row_data.append(row)

4 - Résultats Créer une table de données et structuré

In [22]:
df_paris = pd.DataFrame(row_data)
df_paris

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1er,Louvre,183,74 286,38 926,16 888,17 745,17 308,16 545,16 395,40 593,21 271,9 228,9 697,9 458,9 041
1,2e,Bourse,99,73 578,43 857,19 585,21 259,23 009,20 796,21 042,74 321,44 300,19 783,21 474,23 241,21 006
2,3e,Temple,117,89 687,65 312,34 248,34 721,35 652,35 049,34 389,76 656,55 822,29 272,29 676,30 472,29 956
3,4e,Hôtel-de-Ville,160,95 003,66 621,30 675,28 268,28 012,27 146,28 370,59 377,41 638,19 172,18 211,17 507,16 966
4,5e,Panthéon,254,96 689,106 443,58 849,61 475,60 938,59 333,59 631,38 067,41 907,23 169,24 203,23 991,23 359
5,6e,Luxembourg,215,90 288,88 200,44 919,45 278,43 451,42 428,41 976,41 994,41 023,20 893,21 060,20 210,19 734
6,7e,Palais-Bourbon,409,78 553,104 412,56 985,56 612,57 974,54 133,52 193,19 206,25 529,13 933,13 842,14 175,13 235
7,8e,Élysée,388,75 796,80 827,39 314,39 088,41 280,36 694,37 367,19 535,20 832,10 132,10 074,10 639,9 457
8,9e,Opéra[note 1],218,103 767,102 287,55 838,58 497,60 139,59 408,60 071,47 600,46 921,25 614,26 833,27 587,27 251
9,10e,"Entrepôt, anciennement Enclos Saint-Laurent",289,135 392,129 179,89 612,92 082,95 394,91 770,90 836,46 848,44 699,31 008,31 862,33 008,31 754


Get & Set Foursquare credentials

In [23]:
CLIENT_ID = '5CMJHFWSA1RHJGPGHBV3NM15NR4J1Z0ACEGYVHIGLLY1KCH5' # your Foursquare ID
CLIENT_SECRET = 'YHUIOAL1GG10GR1C3LBBXFVTI1FBRWWCXL2JO3BPTZIAROP1' # your Foursquare Secret
VERSION = '20200731' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5CMJHFWSA1RHJGPGHBV3NM15NR4J1Z0ACEGYVHIGLLY1KCH5
CLIENT_SECRET:YHUIOAL1GG10GR1C3LBBXFVTI1FBRWWCXL2JO3BPTZIAROP1


5- Commentaires de résultats

Le résultat de la répartition de la population affiché en résultat présente en 1er colonne - les numéro d'arrondissement; en 2e- le nom des arrondissements; en 3e colonne - la superficie ; de la 4e à la 10e colonne la population par année difficle à structurer et de la 11e à la dernière - la densité d'habitants par km2.

6- Conclusion

Ce rapport synthétique non approfondit présente un peu, la répartition de la population parisienne dans le temps pouvant aider à la prise de décision des investisseurs immobiliers intérêssés par la ville.